In [1]:
import pandas as pd # python에서 데이터를 조작할 때 쓰는 툴
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

train_df = pd.read_pickle("soma_goods_train.df")
vectorizer = CountVectorizer()

In [2]:
# set d_list, cate_list

title_list = []
cate_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1']])
    title_list.append(each[1]['name'])
    cate_list.append(cate)

In [3]:
# make cate_dict

cate_dict = dict(zip(list(set(cate_list)),range(len(set(cate_list))))) # 카테고리 별로 id를 만들어주는 코드.
joblib.dump(cate_dict,'cate_dict.dat')

['cate_dict.dat']

In [4]:
# make y_list

y_list = []
for i in range(0,10000) :
    y_list.append(cate_dict[cate_list[i]])

In [5]:
# define removeStopWords

mapping = { 'A/S':'AS', u'현대백화점':u'백화점', u'롯데백화점':u'백화점', u'대구백화점':u'백화점', 'IntelIntel':'Intel', 'IBMIBM':'IBM'}
stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인", u"하나 더", u"하나더", u"사은품", u"중고",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른", u"저렴한", u"새로운",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", 
              u"비씨카드", u"우리카드", u"제휴카드", u"하나카드", u"씨티카드",
              u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)
    
def removeStopWords(word) :
     # replace
    for before, after in mapping.iteritems():
        word = word.replace(before, after)
    #remove
    for each in remove_list :
        word = word.replace(each," ")
    return word

In [6]:
# define splitMorph using konlpy

from konlpy.tag import Twitter
from konlpy.utils import pprint
twit = Twitter()

def splitMorph(word) :
    morphs = twit.pos(word)
    
    words = ""
    for morph in morphs :
        word = morph[0]
        words += " " + word
    use_nouns = True
    if use_nouns :
        nouns = twit.nouns(word)
        for noun in nouns :
            words += " " + noun
    return words

In [7]:
# JSON result 미리 읽어오기

list_caffe = []
list_caffe = joblib.load('list_caffe.dat')
length = len(list_caffe)
print length

10000


In [8]:
# remove all stop words
# And split all morphs

length = len(title_list)
for i in range(0, length):
    title_list[i] = removeStopWords(title_list[i])
    title_list[i] = splitMorph(title_list[i])

In [9]:
########### append string

size = 10000
use_caffe = True
if use_caffe :
    idx1 = 0
    idx2 = 0
    for num in range(0, size):
        res = list_caffe[num]['result']
        
        if res[0] :
            
            title = title_list[num]
            for each in res[2] :
                title += " " + each[0].replace(" ", "") + " " + each[0].replace(" ", "")
                if float(each[1]) > 0.7 :
                    title += " " + each[0]
                if float(each[1]) > 1.0 :
                    title += " " + each[0]
            title_list[num] = title
                
print title_list[10]
# print train_df[test_num-1:test_num]

 Intel Pentium Dual Core G 2030 LGA 1155 Processor BX 80637 G 2030 toiletry toiletry lotion lotion container container covering covering binder binder


In [10]:
######### training set과 test set 나누기
title_list_test = []
title_list_train = []
y_list_test = []
y_list_train = []
size = 10000
for i in range(0,size) :
    if i % 10 == 0:
        title_list_test.append(title_list[i])
        y_list_test.append(y_list[i])
    else :
        title_list_train.append(title_list[i])
        y_list_train.append(y_list[i])
        
print len(title_list_test)
print len(y_list_train)

1000
9000


In [11]:
##############################################

save = True
if save :
    joblib.dump(title_list_test, "title_list_test.dat")
    joblib.dump(title_list_train, "title_list_train.dat")
    joblib.dump(y_list_test, "y_list_test.dat")
    joblib.dump(y_list_train, "y_list_train.dat")
        

In [1]:
#############################################
# load
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

title_list_test = joblib.load("title_list_test.dat")
title_list_train = joblib.load("title_list_train.dat")
y_list_test = joblib.load("y_list_test.dat")
y_list_train = joblib.load("y_list_train.dat")

vectorizer = CountVectorizer()

In [2]:
# create x_list
x_list = vectorizer.fit_transform(title_list_train)
print x_list.shape

(9000, 16070)


In [3]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
import numpy as np
#svc_param = {'C':np.logspace(-2,0,20)} 
svc_param = {'C':np.logspace(-2,0,5)} 
gs_svc = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4) 

In [4]:
gs_svc.fit(x_list, y_list_train)

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' w

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.03162,  0.1    ,  0.31623,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [5]:
print gs_svc.best_params_, gs_svc.best_score_
# overfitting underfitting 때문에.
clf = LinearSVC(C=gs_svc.best_params_['C'])
# C 값이 overfitting을 결정한다.
clf.fit(x_list,y_list_train)  # fit 하면 학습이 됨.

{'C': 1.0} 0.981333333333


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [6]:
# test
size = len(title_list_test)
sum = 0
for num in range (0, size):
    title = title_list_test[num]
    vec = vectorizer.transform([title])
    pred = clf.predict(vec)[0]
    answer = y_list_test[num]
    if pred == answer:
        sum = sum + 1
print float(sum)/float(size)

0.987


In [7]:
# 만든 모델 저장
joblib.dump(clf,'classify_test.model',compress=3)
joblib.dump(vectorizer,'vectorizer_test.dat',compress=3)

['vectorizer_test.dat']